In [17]:
from google.colab import drive
drive.mount('/content/drive')
import os
try:
  os.chdir("./drive/My Drive/SpacyToolKit")
except: pass
#  проверяем наличие файлов
directory = os.listdir()
directory

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['0.ipynb', 'SpacyToolKit', 'resume_train.csv', 'vacs_train_5000.csv']

In [18]:
if "SpacyToolKit" not in directory:
    !git clone https://github.com/Lednik7/SpacyToolKit.git

In [23]:
import pandas as pd
df = pd.read_csv("vacs_train_5000.csv")[:100]
df.head()

,description,key_skills
0,Обязанности: Составление сметы по ремонтно-от...,Ответственность | Коммуникабельность
1,В сеть магазинов детской и подростковой обуви ...,Консультативные продажи | Активные продажи | 1...
2,Менеджер по работе с клиентамиТребования: хол...,Холодные звонки | Холодные продажи | Ведение п...
3,«1С:Северо-Запад» - официальный представител...,Работа в команде | 1С: Документооборот | 1С: П...
4,"Компания ООО ""1С:Бухобслуживание. Региональный...",Активные продажи | Навыки презентации | Поиск ...


In [117]:
from SpacyToolKit import separator
from SpacyToolKit.utils import preprocessing
from SpacyToolKit.training import create_blank, begin_training
from SpacyToolKit.model_selection import train_test_val_split
from SpacyToolKit.metrics import evaluate

In [122]:
y = separator(df["key_skills"], sep=" | ")
X = df["description"]
#  coef регулирует долю всех возможных values из key_skills(елси all_tasks=True)
#  При coef = 1 используются все values, это сильно сказывается на времени выполнения
train = preprocessing(X, y, "Skills", coef=1, all_tasks=False)

100%|██████████| 100/100 [00:00<00:00, 1263.12it/s]


coef is ignored because you are using all_tasks=False


In [123]:
Train, Test, Val = train_test_val_split(train)
nlp, ner = create_blank(Train, lang="en") #  создадим пустой шаблон
#  будем сохранять модель после каждой итерации
nlp = begin_training(nlp, Train, Val, n_iter=2, save_iter=True)

  0%|          | 0/40 [00:00<?, ?it/s]


Save models to ./iterations/

Iterations: 1


  0%|          | 0/40 [00:00<?, ?it/s]


{'Skills': {'p': 0.0, 'r': 0.0, 'f': 0.0}}
Losses: {'ner': 1898.8229281974204}

Iterations: 2


100%|██████████| 40/40 [00:03<00:00, 11.64it/s]



{'Skills': {'p': 0.0, 'r': 0.0, 'f': 0.0}}
Losses: {'ner': 184.91926496245287}
Done


In [124]:
# f - f1 score
# p - precision
# r - recall
evaluate(nlp, Test)

{'Skills': {'f': 0.0, 'p': 0.0, 'r': 0.0}}